In [1]:
import time, datetime, json, logging, configparser, traceback, math
from dateutil.parser import parse as dtparser
import pandas as pd
from database import Engine
from sqlalchemy.orm import Session
from sqlalchemy import Column, String, Integer, Float, TIMESTAMP, DateTime
from sqlalchemy.orm import declarative_base
from sqlalchemy.sql import func
from requests.exceptions import ConnectionError

from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.forexlabs as labs
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.transactions as trans

# from models import Order, Trade, OrderFillTransaction, Alerts
# from order_management import handle_alert

In [2]:
file_handler = logging.FileHandler('log.log')
logging.basicConfig(handlers=[file_handler], level=logging.INFO, format='%(asctime)s %(levelname)s %(name)s %(message)s')
logger = logging.getLogger(__name__)

In [3]:
cfg = configparser.RawConfigParser()
cfg.read(filenames='credentials.ini')

access_token = cfg['keys']['demo_key']
accountID = cfg['keys']['accountID']
client = API(access_token=access_token)

oins = pd.read_csv('instruments.csv')


In [4]:
symbol_mapper = {
    'EURUSD':'EUR_USD',
    'US30USD':'US30_USD',
    'XAUUSD':'XAU_USD',
    'XAGUSD':'XAG_USD',
    'US30' : 'US30_USD',
    'BCOUSD' : 'BCO_USD',
}

In [26]:
oins[oins.type == 'CFD']

,name,type,displayName,pipLocation,displayPrecision,tradeUnitsPrecision,minimumTradeSize,maximumTrailingStopDistance,minimumTrailingStopDistance,maximumPositionSize,...,ASSET_CLASS,longRate,shortRate,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY,SATURDAY,SUNDAY
3,BCO_USD,CFD,Brent Crude Oil,-2,3,0,1.0,100,0.0500,0,...,COMMODITY,-0.0050,-0.0450,1,1,1,1,1,1,1
14,UK10YB_GBP,CFD,UK 10Y Gilt,-2,3,0,1.0,100,0.0500,0,...,BOND,0.0160,-0.0660,1,1,1,1,1,1,1
17,JP225_USD,CFD,Japan 225,0,1,1,0.1,10000,5.0000,0,...,INDEX,-0.0756,0.0256,1,1,1,1,3,0,0
20,SG30_SGD,CFD,Singapore 30,-1,2,1,0.1,1000,0.5000,0,...,INDEX,-0.0650,0.0150,1,1,1,1,3,0,0
21,XPT_USD,CFD,Platinum,-2,3,0,1.0,100,0.0500,0,...,COMMODITY,-0.0430,-0.0070,1,1,1,1,1,1,1
25,DE30_EUR,CFD,Germany 30,0,1,1,0.1,10000,5.0000,0,...,INDEX,-0.0565,0.0065,1,1,1,1,3,0,0
28,CN50_USD,CFD,China A50,0,1,1,0.1,10000,5.0000,0,...,INDEX,-0.0756,0.0256,1,1,1,1,3,0,0
36,TWIX_USD,CFD,Taiwan Index,0,1,1,0.1,10000,5.0000,0,...,INDEX,-0.0756,0.0256,1,1,1,1,3,0,0
39,WTICO_USD,CFD,West Texas Oil,-2,3,0,1.0,100,0.0500,0,...,COMMODITY,-0.0320,-0.0180,1,1,1,1,1,1,1
51,ESPIX_EUR,CFD,Spain 35,0,1,1,0.1,10000,5.0000,0,...,INDEX,-0.0565,0.0065,1,1,1,1,3,0,0


In [6]:
def error_handler(func,current_try = 1, *args, **kwargs):
    # wrapper function for handling errors
    try:
        return func(*args, **kwargs)
    except Exception as e:
        #assign exception to the error_type
        print(f'ERROR IN {func} @ {datetime.datetime.now().time().replace(microsecond=0)} : {e}')
        logging.error(f'ERROR IN {func} @ {datetime.datetime.now().time().replace(microsecond=0)} : {e}')
        logging.error(f'FUNCTION ARGUMENTS ARE :{args}, {kwargs}')
        logging.error(traceback.format_exc())
        
        if current_try < 4:
            current_try += 1
            error_handler(func,current_try=current_try, *args, **kwargs)
        
        return

def place_order(price: str, instrument: str, units:str, order_type: str= 'LIMIT', stop_price:str = None, target_price:str = None, trailing_stop_price:str = None, timeInForce: str= 'GTC'):
    
    data = {"order": {
                "price": price,
                "timeInForce": timeInForce,
                "instrument": instrument,
                "units": units,
                "type": order_type,
                "positionFill": "DEFAULT"
                    } }
    if stop_price:
        stops = {"stopLossOnFill": {
                    "timeInForce": 'GTC',
                    "price": stop_price
                        } }
        data['order'].update(stops)
    
    if target_price:
        target = {"takeProfitOnFill": {
                    "timeInForce": 'GTC',
                    "price": target_price
                        } }
        data['order'].update(target)
    
    if trailing_stop_price:
        trailing = {"trailingStopLossOnFill": {
                    "timeInForce": 'GTC',
                    "distance": trailing_stop_price
                        } }
        data['order'].update(trailing)
        
    req = orders.OrderCreate(accountID, data=data)
    response = client.request(req)
    return response

def modify_order(orderID: str, units: str, instrument: str, price: str, order_type: str= 'MARKET', accountID: str= accountID):
	data = {"order": {
                "units": units,
                "instrument": instrument,
                "price": price,
                "type": order_type }
            }
	req = orders.OrderReplace(accountID=accountID, orderID=orderID, data=data)
	response = client.request(req)
	return response

def cancel_order(orderID: str, accountID = accountID):
    req = orders.OrderCancel(accountID= accountID, orderID= orderID)
    cancelled_order = client.request(req)
    return cancelled_order

def get_order_details(orderID, accountID = accountID):
    req = orders.OrderDetails(accountID= accountID, orderID= orderID)
    response = client.request(req)
    return response

def get_order_list(accountID = accountID):
    req = orders.OrderList(accountID)
    response  = client.request(req)
    return response

def get_pending_orders(accountID = accountID):
    req = orders.OrdersPending(accountID)
    response  = client.request(req)
    return response



def get_candles(instrument: str, _from: datetime= None, _to: datetime= None, gran: str= 'M5'):    
    params = {"granularity": gran}
    if _from:
        params.update({'from':_from})
    if _to:
        params.update({'to':_to})

    req = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = client.request(req)
    return response

def get_instrument_position_book(instrument: str):
    req = instruments.InstrumentsPositionBook(instrument=instrument, params={})
    response = client.request(req)
    return response

def get_instrument_order_book(instrument: str):
    req = instruments.InstrumentsOrderBook(instrument=instrument, params={})
    response = client.request(req)
    return response



def get_open_positions(accountID = accountID):
    req = positions.OpenPositions(accountID=accountID)
    response = client.request(req)
    return response

def close_position(instrument: str, data: dict, accountID: str= accountID):
    ''' Close Open Position for given symbol and quantity for selected Account \n
    data = {'longUnits' : 'ALL} \n
    data = {'shortUnits : '100'}
    '''
    req = positions.PositionClose(accountID=accountID, instrument=instrument, data=data)
    response = client.request(req)
    return response

def get_position_details(instrument: str, accountID: str= accountID):
    ''' Get details of position for given symbol and Account '''
    req = positions.PositionDetails(accountID=accountID, instrument=instrument)
    response = client.request(req)
    return response

def get_position_list(accountID: str= accountID):
    ''' Get list of all positions for given Account'''
    req = positions.PositionList(accountID=accountID)
    response = client.request(req)
    return response



def get_prices(instruments: list, accountID: str= accountID):
    params ={
            "instruments": ','.join(instruments)
            }
    req = pricing.PricingInfo(accountID=accountID, params=params)
    response = client.request(req)
    return response

def price_streams(instruments: list, max_records: int= 10, accountID: str= accountID):
    params ={
            "instruments": ','.join(instruments)
            }
    req = pricing.PricingStream(accountID=accountID, params=params)
    response = client.request(req)

    maxrecs = 10
    for ticks in req:
        yield ticks
        maxrecs -= 1
        if maxrecs == 0:
            req.terminate("maxrecs records received")



def get_open_trades(accountID: str= accountID):
    req = trades.OpenTrades(accountID= accountID)
    response = client.request(req)
    return response

def close_trade(units: str, tradeID: str, accountID: str= accountID):
    data = {"units":str(units)}
    req = trades.TradeClose(data=data, tradeID=str(tradeID), accountID= str(accountID))
    response = client.request(req)
    return response

def get_trade_details(tradeID: str, accountID: str= accountID):
    req = trades.TradeDetails(tradeID=str(tradeID), accountID= str(accountID))
    response = client.request(req)
    return response

def get_trade_list(instrument: str, accountID: str= accountID):
    # not working properly, instead of giving all trades list, giving only open trades
    params = {"instrument": instrument}
    req = trades.TradesList(params= params, accountID= str(accountID))
    response = client.request(req)
    return response

def add_trade_comment(tradeID: str, accountID: str= accountID, comment: str= None, id: str= None):
  data = {
    "clientExtensions": {
      "comment": comment,
      "id": id } }
  req = trades.TradeClientExtensions(accountID= accountID, tradeID= 93, data= data)
  response = client.request(req)
  return response



def get_account_summary(accountID: str= accountID):
    req = accounts.AccountSummary(accountID)
    response = client.request(req)
    return response

def get_account_details(accountID: str= accountID):
    req = accounts.AccountDetails(accountID)
    response = client.request(req)
    return response



def get_transaction_details(trasactionID:str, accountID: str= accountID):
    req = trans.TransactionDetails(accountID=accountID, transactionID= str(trasactionID))
    response = client.request(req)
    return response

def get_transaction_details_by_range(_from: str, _to: str, accountID: str= accountID):
    params = {"from" : _from, 'to' : _to}
    req = trans.TransactionIDRange(accountID=accountID, params=params)
    response = client.request(req)
    return response

def get_transaction_list(accountID: str= accountID):
    req = trans.TransactionList(accountID=accountID)
    response = client.request(req)
    return response

def get_transactions_since_id(id: str, accountID: str= accountID):
    params = {'id' : id}
    req = trans.TransactionsSinceID(accountID = accountID, params = params)
    response = client.request(req)
    return response

In [7]:
Base = declarative_base()

class error_log(Base):
    __tablename__ = 'error_log'
    sr = Column(Integer, primary_key=True)
    timestamp = Column(TIMESTAMP, server_default=func.now())
    utc_timestamp = Column(DateTime)
    error_type = Column(String(255), nullable=False)
    error_message = Column(String(255), nullable=False)

class Alerts(Base):
    __tablename__ = 'alert'
    sr = Column(Integer, primary_key=True)
    timestamp = Column(TIMESTAMP, server_default=func.now())
    alert_type = Column(String(50), nullable=False)
    alert_message = Column(String(500), nullable=False)
    timeframe = Column(String(10))
    exchange = Column(String(10))
    alert_symbol = Column(String(50))
    mapped_symbol = Column(String(50))
    entry_side = Column(String(10))
    order_size = Column(String(10))
    current_position = Column(String(20))
    trade_multiplier = Column(String(10))
    entry_price = Column(String(15))
    target = Column(String(15))
    stop_loss = Column(String(15))
    trailing_stop = Column(String(15))
    trades_open_ids = Column(String(100))
    trades_closed_ids = Column(String(100))
    

class Order(Base):
    __tablename__ = 'order'
    sr = Column(Integer, primary_key=True)
    timestamp = Column(TIMESTAMP, server_default=func.now())
    alert_sr = Column(Integer)
    type = Column(String(30))
    instrument = Column(String(30))
    timeInForce = Column(String(10))
    
    order_id = Column(String(10))
    order_time = Column(DateTime)
    order_units = Column(String(20))
    order_price = Column(String(20))
    reason = Column(String(300))
    target_price = Column(String(20))
    stop_loss_price = Column(String(20))
    trailing_stop_price = Column(String(20))
    
    trade_price = Column(String(20))
    trade_id = Column(String(10))
    trade_time = Column(DateTime)
    trade_units = Column(String(10))
    trade_price = Column(String(20))
    trade_state = Column(String(10))
    marginUsed = Column(String(30))
    
    accountID = Column(String(50))
    userID = Column(String(20))
    batchID = Column(String(20))
    requestID = Column(String(50))
    triggerCondition = Column(String(20))
    partialFill = Column(String(20))
    positionFill = Column(String(30))

class Trade(Base):
    __tablename__ = 'trades'
    sr = Column(Integer, primary_key= True)
    timestamp = Column(TIMESTAMP, server_default=func.now())
    time = Column(DateTime)
    orderID = Column(Integer)
    price = Column(Float)
    tradeID = Column(Integer)
    units = Column(Integer)
    guaranteedExecutionFee = Column(Float)
    quoteGuaranteedExecutionFee = Column(Float)
    halfSpreadCost = Column(Float)
    initialMarginRequired = Column(Float)

class OrderFillTransaction(Base):
    __tablename__ = 'orderfilltransactions'
    
    sr = Column(Integer, primary_key=True)
    timestamp = Column(TIMESTAMP, server_default=func.now())
    id = Column(Integer)
    accountID = Column(String(50))
    userID = Column(Integer)
    batchID = Column(String(50))
    requestID = Column(String(50))
    time = Column(DateTime)
    type = Column(String(50))
    orderID = Column(String(50))
    instrument = Column(String(50))
    units = Column(String(10))
    requestedUnits = Column(String(10))
    price = Column(Float)
    pl = Column(Float)
    quotePL = Column(String(50))
    financing = Column(Float)
    baseFinancing = Column(String(50))
    commission = Column(Float)
    accountBalance = Column(Float)
    gainQuoteHomeConversionFactor = Column(Float)
    lossQuoteHomeConversionFactor = Column(Float)
    guaranteedExecutionFee = Column(Float)
    quoteGuaranteedExecutionFee = Column(String(50))
    halfSpreadCost = Column(Float)
    fullVWAP = Column(Float)
    reason = Column(String(500))
    initialMarginRequired = Column(Float)
    closeoutBid = Column(Float)
    closeoutAsk = Column(Float)
    closeoutTimestamp = Column(DateTime)
    bidPrice = Column(Float)
    bidLiquidity = Column(Integer)
    askPrice = Column(Float)
    askLiquidity = Column(Integer)
    gainBaseHomeConversionFactor = Column(Float)
    lossBaseHomeConversionFactor = Column(Float)

In [8]:
Base.metadata.create_all(Engine)

In [9]:
def save_alert(alert, alert_type):
    if alert['Side'] == 'buy':
        order_size = str(round(float(alert['Order_Size']), 1))
        
    else:
        order_size = str(round(-float(alert['Order_Size']), 1))
        
    _alert = Alerts(
        alert_type    = alert_type,
        alert_message = str(alert),
        timeframe     = str(alert['Timeframe']),
        exchange      = str(alert['Exchange']),
        alert_symbol  = str(alert['Symbol']),
        mapped_symbol = symbol_mapper[alert['Symbol']],
        entry_side    = alert['Side'],
        order_size    = order_size,
        current_position = str(alert['Current_Position']),
        trade_multiplier = str(alert['Trade_Mulitplier']),
        entry_price   = str(alert['Entry']),
        target        = str(alert['Target']),
        stop_loss     = str(alert['Stop_Loss']),
        trailing_stop = str(alert['Profit_Trail'])
        )

    with Session(Engine) as session:
        session.add(_alert) #add alert in the database
        session.commit() #save changes made in database
        alert_sr = _alert.sr
    return alert_sr

def save_filled_order(order_details, alert_sr):
    
    order_data = order_details[0].copy()
    trade_data = order_details[1].copy()
    data = {}
    
    data['alert_sr'] = alert_sr
    data['type'] = order_data.get('type')
    data['instrument'] = order_data.get('instrument')
    data['timeInForce'] = order_data.get('timeInForce')
    
    data['order_id'] = order_data.get('id')
    data['order_time'] = dtparser(order_data.get('time'))
    data['order_units'] = order_data.get('units')
    data['order_price'] = order_data.get('price')
    data['reason'] = order_data.get('reason')

    data['trade_id'] = trade_data.get('id')
    data['trade_units'] = trade_data.get('currentUnits')
    data['trade_price'] = trade_data.get('price')
    data['trade_state'] = trade_data.get('state')

    if trade_data.get('state') == 'OPEN':
        data['marginUsed'] = trade_data.get('marginUsed')
    
    if trade_data.get('openTime'):
        data['trade_time'] = dtparser(trade_data.get('openTime'))

    if trade_data.get('time'):
        data['trade_time'] = dtparser(trade_data.get('time'))

    if order_data.get('takeProfitOnFill'):
        data['target_price'] = order_data.get('takeProfitOnFill')['price']
        
    if order_data.get('stopLossOnFill') :
        data['stop_loss_price'] = order_data.get('stopLossOnFill')['price']
        
    if order_data.get('trailingStopLossOnFill') :
        data['trailing_stop_price'] = order_data.get('trailingStopLossOnFill')['distance']
    
    data['accountID'] = order_data.get('accountID')
    data['userID'] = order_data.get('userID')
    data['batchID'] = order_data.get('batchID')
    data['requestID'] = order_data.get('requestID')
    data['triggerCondition'] = order_data.get('triggerCondition')
    data['partialFill'] = order_data.get('partialFill')
    data['positionFill'] = order_data.get('positionFill')

    _order = Order(**data)

    with Session(Engine) as session:
        session.add(_order)
        session.commit()
        order_sr = _order.sr
    
    return order_sr

In [10]:
def handle_entry_alert(entry_alert, order_type = 'MARKET'):

    symbol = symbol_mapper[entry_alert['Symbol']]
    symbol_token = oins[oins.name == symbol].iloc[0]
    precision = symbol_token['displayPrecision']
    unit_precision = symbol_token['tradeUnitsPrecision']
    
    timeInForce = 'IOC' if order_type == 'MARKET' else 'GTC'
    
    if entry_alert['Side'] == 'buy':
        order_size = str(round(float(entry_alert['Order_Size']) + float(entry_alert['Current_Position']), unit_precision) )
        alert_entry_price = str(round(float(entry_alert['Entry']) * 1.0005, precision))

    elif entry_alert['Side'] == 'sell':
        order_size = str(round(-float(entry_alert['Order_Size']) + float(entry_alert['Current_Position']), unit_precision) )
        alert_entry_price = str(round(float(entry_alert['Entry']) * 0.9995, precision))
    
    stop_loss = str(round(float(entry_alert['Stop_Loss']), precision)) if not math.isnan(float(entry_alert['Stop_Loss'])) else None
    target = str(round(float(entry_alert['Target']), precision)) if not math.isnan(float(entry_alert['Target'])) else None
    trailing_stop = str(round(float(entry_alert['Profit_Trail']), precision)) if not math.isnan(float(entry_alert['Profit_Trail'])) else None
    if trailing_stop:
        distance = str(round(abs(float(alert_entry_price) - float(trailing_stop)), precision))
    else:
        distance = None
    
    order_details = []
    for i in range(int(entry_alert['Trade_Mulitplier'])):
        order_detail = error_handler(place_order, instrument= symbol, units= order_size, order_type= order_type, price= alert_entry_price, stop_price= stop_loss, target_price= target, trailing_stop_price= distance,  timeInForce= timeInForce)
        order_details.append(order_detail)
        time.sleep(1)

    return order_details

In [11]:
def check_order_status(order_detail, current_try = 1):
    
    order_id = order_detail['orderCreateTransaction']['id']
    ord_det = error_handler(get_order_details, orderID = order_id)
    
    if ord_det.get('order'):
        
        if ord_det['order']['state'] == 'FILLED':
            if ord_det['order'].get('tradeOpenedID'):
                trd_det = error_handler(get_trade_details, tradeID = ord_det['order']['tradeOpenedID'])
                if trd_det.get('trade') :
                    return 'FILLED', trd_det['trade']
                    
            if ord_det['order'].get('fillingTransactionID'):
                trd_det = error_handler(get_transaction_details, trasactionID = ord_det['order']['fillingTransactionID'])
                if trd_det.get('transaction') :
                    return 'FILLED', trd_det['transaction']

        else:
            return ord_det['order']['state'], ord_det
        


In [12]:
def save_order_trade_details(order_details, alert_id):
    trade_details, pending_orders, cancelled_orders = [], [], []

    for _order in order_details:
        _order_status, trade_detail = check_order_status(_order)

        if _order_status == 'FILLED':
            trade_details.append((_order.get('orderCreateTransaction'), trade_detail))
        elif _order_status == 'PENDING':
            pending_orders.appned(trade_detail)
        elif _order_status == 'CANCELLED':
            cancelled_orders.append(trade_detail)
    
    data_ids = []
    for trade in trade_details:
        saved_data_sr = save_filled_order(trade, alert_id)
        data_ids.append(saved_data_sr)
    
    return data_ids

In [13]:
def handle_position_exit(alert, current_try: int= 1):

    closed_trades = []

    if float(alert['Current_Position']) == 0 :
        return closed_trades
    
    symbol = symbol_mapper[alert['Symbol']]
    open_positions = error_handler(get_open_positions)['positions']
    
    if open_positions == None and current_try < 4 :
        current_try += 1
        handle_position_exit(alert, current_try= current_try)
        

    for position in open_positions:
        if position['instrument'] == symbol:
            
            if alert['Side'] == 'buy':
                qty = float(position['short']['units'])
                if qty != 0 :
                    for _id in position['short']['tradeIDs']:
                        with Session(Engine) as session:
                            _orders = session.query(Order.trade_id).filter_by(instrument = symbol).all()
                            for _ord in _orders:
                                if _ord[0] == _id:
                                    trd_detail = error_handler(get_trade_details, tradeID = _id)['trade']
                                    trade_details = error_handler(close_trade, units = abs(float(trd_detail['currentUnits'])), tradeID = _id)
                                    closed_trades.append(trade_details)

                elif qty > 0 and abs(qty) == alert['Order_Size']:
                    print('there is some error, either last position was not closed properly using algo.')

            else :
                qty = float(position['long']['units'])
                if qty != 0:
                    for _id in position['long']['tradeIDs']:
                        with Session(Engine) as session:
                            _orders = session.query(Order.trade_id).filter_by(instrument = symbol).all()
                            for _ord in _orders:
                                if _ord[0] == _id:                        
                                    trd_detail = error_handler(get_trade_details, tradeID = _id)['trade']
                                    trade_details = error_handler(close_trade, units = abs(float(trd_detail['currentUnits'])), tradeID = _id)
                                    closed_trades.append(trade_details)

                elif qty < 0 and abs(qty) == alert['Order_Size']:
                    print('there is some error, either last position was not closed properly using algo.')

    return closed_trades

In [14]:
def handle_alert(alert):
    
    closed_trades = error_handler(handle_position_exit, alert)
    
    order_details = error_handler(handle_entry_alert, alert)
    
    alert_id = error_handler(save_alert, alert=alert, alert_type = 'entry_alert')

    order_ids = error_handler(save_order_trade_details, order_details = order_details, alert_id = alert_id)

    close_ids = error_handler(save_order_trade_details, order_details = closed_trades, alert_id = alert_id)
    
    try:
        logging.warning(f'Order SR : {order_ids}, Alert SR : {alert_id}, New Order Details : {order_details}, Closed Trade Details : {closed_trades}')
    except:
        pass
    
    return alert_id, order_ids, order_details, closed_trades

In [15]:
entry_alert = {'Side': 'buy', 'Order_Size': '100', 'Timeframe': '1', 'Exchange': 'FX', 'Symbol': 'US30', 'Current_Position': '-50', 'Trade_Mulitplier': '1', 'Entry': '33825', 'Stop_Loss': 'NaN', 'Target': 'NaN', 'Profit_Trail': 'NaN'}
exit_alert = {'Side': 'sell', 'Order_Size': '100', 'Timeframe': '1', 'Exchange': 'FX', 'Symbol': 'US30', 'Current_Position': '50', 'Trade_Mulitplier': '1', 'Entry': '33825', 'Stop_Loss': 'NaN', 'Target': 'NaN', 'Profit_Trail': 'NaN'}

In [15]:
# entry_details = handle_alert(entry_alert)

In [16]:
# exit_details = handle_alert(exit_alert)

In [17]:
# close_position('US30_USD', {'longUnits': 'ALL'}) #, 'shortUnits': 'ALL'

In [18]:
# try:
#     print(get_open_positions())
# except Exception as e :
#     print(type(e))
#     if type(e) == ConnectionError:
#         print('done')
#     print(e)

In [19]:
error_handler(get_open_positions)

{'positions': [{'instrument': 'EUR_USD',
   'long': {'units': '50',
    'averagePrice': '1.07488',
    'pl': '-2757.0177',
    'resettablePL': '-2757.0177',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'tradeIDs': ['4969'],
    'unrealizedPL': '-0.0120'},
   'short': {'units': '-50',
    'averagePrice': '1.07490',
    'pl': '-33.0855',
    'resettablePL': '-33.0855',
    'financing': '0.0000',
    'dividendAdjustment': '0.0000',
    'guaranteedExecutionFees': '0.0000',
    'tradeIDs': ['4981'],
    'unrealizedPL': '0.0060'},
   'pl': '-2790.1032',
   'resettablePL': '-2790.1032',
   'financing': '0.0000',
   'commission': '0.0000',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '-0.0060',
   'marginUsed': '2.6868'}],
 'lastTransactionID': '4997'}